## Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación\*

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo**

### Alumnos:

* Pablo Moreira Garcia
* Ruben Martinez Sotoca

En esta primera práctica, estudiaremos el proceso KDD siguiendo los pasos necesarios y comprendiendo tanto su utilidad como su correcta utilización. Los pasos que seguiremos son:
* Carga de Datos
* Análisis Exploratorio
* Preprocesamiento de datos
* Validación de los modelos

Así, cuando acabemos la práctica, abremos utilizado dos bases de datos (Pima y Wisconsin), para comprender de forma más profunda el proceso de generación, mejora y validación de modelos.

# 1. Preliminares

El primer paso consiste en importar las librerías que nos serán útiles en los distintos apartados de la práctica.

In [ ]:
# Third party
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as mpl
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np


# Local application
import utilidades_practica_1_ordinaria as utils

También es necesario, antes de comenzar a trabajar, definir una semilla que nos asegure que la distribución aleatoria de los datos no varíe de una ejecución a otra.

In [ ]:
seed = 27912

# 2. Acceso y almacenamiento de datos

Vamos a usar dos bases de datos en esta práctica:

Una es la base de datos Breast Cancer Wisconsin (Diagnostic) Data Set. La cual, mediante valores médicos relacionados, permite diagnosticar a un paciente con un cancer maligno o benigno. Las variables médicas que estudiaremos son:

* `Area_mean`: Media del area
* `Diagnosis`: El diagnostico final (M = maligno, B = benigno)
* `Smoothness_mean`: Media de las variaciones locales en longitudes de radio
* `Texture_mean`: Desviación estándar de los valores de la escala de grises.
* `Radius_mean`: Media de distancias desde el centro a puntos del perimetro 
* `Perimeter_mean`: Tamaña del centro del tumor.
* `Concave_points_mean`: Número medio de porciones cóncavas del contorno.
* `Compactness_mean`: Media del perimetro^2 / area - 1.0
* `Concavity_mean`: Media de la gravedad de las porciones cóncavas del contorno.
* `Symmetry_mean`: Media de las simetrias
* `Fractal_Dimension_mean`: Media de "coastline approximation" - 1

Asociada a cada variable, tenemos también tenemos su error estándar y su peor caso.

A partir de estas variables, se guarda la conclusión en la variable objetivo "Outcome" bajo el valor de M o B. Si el resultado es M el tumor es maligno, si es B, benigno.

La otra base de datos que usaremos es la Pima Indians Database. Este conjunto de datos sirve para, partiendo de las variables que vemos a continuación, predecir si una paciente sufre de diabetes o no.

* `Glucose`: Concentración de glucosa en plasma sanguíneo de la paciente.
* `Pregnancies`: Número de embarazos de la paciente.
* `insulin`: Cantidad de insulina en sangre de la paciente (en mu Insulina/ml).
* `BloodPressure`: Presión diastólica arterial de la paciente (en mm Hg).
* `SkinThickness`: Grosor de la piel en el triceps de la paciente (en mm).
* `DiabetesPedigreeFunction`: Función del historial de diabetes en la familia del paciente.
* `BMI`: Índice de Masa Corporal de la paciente (en kg/m^2).
* `Age`: Edad de la paciente.

Estudiando las variables, se obtiene un diagnóstico que se almacena en `Outcome` con el valor 0 o 1. Si el valor es 1, la paciente tiene diabetes y si es 0, no tiene.

A continuación, incluimos las bases de datos, indicando en cada una cual es la variable objetivo.

In [ ]:
filepath = "../input/breast-cancer-wisconsin-data/data.csv"

index = "id"
target = "diagnosis"

wisconsin_data = utils.load_data(filepath, index, target)

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

pima_data = utils.load_data(filepath, index, target)

En el caso de wisconsin, debemos indicar "diagnosis" como la variable objetivo y la variable "id", como índide.
En Pima, sin embargo, como no tenemos índice en la base de datos, indicamos el index como None y "Outcome" como variable objetivo.

Una vez hemos introducido los datos, utilizando la función sample (que muestra un conjunto aleatorio de filas), podemos comprobar que se han introducido correctamente.

In [ ]:
wisconsin_data.sample(5, random_state=seed)

In [ ]:
pima_data.sample(5, random_state=seed)

Es importante para el proceso diferenciar entre el conjunto de variables predictoras y las variables objetivo.

In [ ]:
(X_wisconsin, y_wisconsin) = utils.divide_dataset(wisconsin_data, target="diagnosis")

In [ ]:
(X_pima, y_pima) = utils.divide_dataset(pima_data, target="Outcome")

Podemos volver a utilizar la función sample, para comprobar que la partición se ha hecho correctamente.

In [ ]:
X_wisconsin.sample(5, random_state=seed)

In [ ]:
X_pima.sample(5, random_state=seed)

Y continuamos con la variable clase:

In [ ]:
y_wisconsin.sample(5, random_state=seed)

In [ ]:
y_pima.sample(5, random_state=seed)

A continuación, es necesario dividir el conjunto de datos en dos. Por un lado, creamos un conjunto de entrenamiento y otro de prueba.
El primero lo utilizaremos para entrenar nuestro modelo y mejorar su rendimiento, y el segundo se dejará intacto para, cuando haya terminado el proceso de entrenamiento, utilizarlo para comprobar la eficacia del modelo.
En este caso, vamos a dividir los conjuntos siguiendo la proporción más típica: un 70% para el de entrenamiento y un 30% para el de prueba.

Utilizaremos la función train_test_split, a la que le indicaremos la proporción ya indicada. Además, incluiremos la semilla y el conjunto de datos a utilizar para la estratificación.

In [ ]:
train_size = 0.7

(X_wisconsin_train, X_wisconsin_test, y_wisconsin_train, y_wisconsin_test) = train_test_split(X_wisconsin, y_wisconsin,
                                                      stratify=y_wisconsin,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
train_size = 0.7

(X_pima_train, X_pima_test, y_pima_train, y_pima_test) = train_test_split(X_pima, y_pima,
                                                      stratify=y_pima,
                                                      random_state=seed,
                                                      train_size=train_size)

De nuevo, vamos a utilizar la función sample para comprobar que la separación se ha realizado correctamente.

In [ ]:
X_pima_train.sample(5, random_state=seed)

In [ ]:
X_wisconsin_test.sample(5, random_state=seed)

In [ ]:
X_pima_train.sample(5, random_state=seed)

In [ ]:
X_pima_test.sample(5, random_state=seed)

Por último, finalizamos con la variable objetivo del conjunto de datos de entrenamiento:

In [ ]:
y_wisconsin_train.sample(5, random_state=seed)

In [ ]:
y_pima_train.sample(5, random_state=seed)

Y prueba:

In [ ]:
y_wisconsin_test.sample(5, random_state=seed)

In [ ]:
y_pima_test.sample(5, random_state=seed)

Volvemos a unir las variables predictoras y las variables clase ya que lo necesitaremos para poder visualizar esta informacion con diferentes graficas a lo largo de la practica, lo hacemos ahora para no tener que ir uniendolas cada vez que nos haga falta y poder facilitar el analisis exploratorio de datos.

In [ ]:
wisconsin_data_train = utils.join_dataset(X_wisconsin_train, y_wisconsin_train)

In [ ]:
pima_data_train = utils.join_dataset(X_pima_train, y_pima_train)

In [ ]:
wisconsin_data_test = utils.join_dataset(X_wisconsin_test, y_wisconsin_test)

In [ ]:
pima_data_test = utils.join_dataset(X_pima_test, y_pima_test)

In [ ]:
wisconsin_data_train.sample(5, random_state=seed)

In [ ]:
pima_data_train.sample(5, random_state=seed)

In [ ]:
wisconsin_data_test.sample(5, random_state=seed)

In [ ]:
pima_data_test.sample(5, random_state=seed)

Se puede ver que la union de los conjuntos se han unido correctamente.

# 3. Análisis exploratorio de datos

En este paso nos vamos a centrar en las características de cada conjunto de datos, estudiando sus variables, los valores que toman y las relaciones que se puedan apreciar de ellas. Para facilitar este estudio, vamos a recurrir a varias funciones gráficas que muestren la información de forma más visual.

El número de casos y variables (respectivamente) del conjunto de datos se puede obtener consultando el atributo `shape`:

In [ ]:
wisconsin_data_train.shape

El conjunto de datos está formado por 398 casos y 32 variables (31 variables predictoras y 1 variable clase).

Podemos ver distinta informacion sobre nuestras bases de datos gracias al metodo `info` como el número de entradas, el número total de variables, la cantidad de `Non-Null` por variable o el tipo de cada una de estas:

In [ ]:
wisconsin_data_train.info(memory_usage=False)

 Segun "info", de las 32 variables, 31 son numericas (float) que coincide con las variables predictoras, y diagnosis, la variable clase (categorica). Y un dato que cabe destacar es que la variable Unnamed: 32 Es una variable despreciable ya que no tiene informacion y es todo Null.

In [ ]:
pima_data_train.shape

Tal y como se puede observar, el conjunto de datos está formado por 537 casos y 9 variables (8 variables predictoras y 1 variable clase).

Para conocer informacion sobre esta base de datos recurrimos al método `info` de nuevo:

In [ ]:
pima_data_train.info(memory_usage=False)

Según la función info, tenemos 8 variables predictoras las cuales son numéricas (float e int) y una variable clase (categorica).

Usando la función categories podemos ver que valores pueden tomar este tipo de variables.

In [ ]:
y_wisconsin_train.cat.categories

In [ ]:
y_pima_train.cat.categories

Ambas variables objetivo toman dos valores, pero la de pima se registra en unos y ceros, y la de wisconsin toma los valores "B" y "M".

### Visualización de las variables

Ahora que conocemos un poco más en detalle las características de las variables, vamos a proceder a representarlas gráficamente. Debemos tener en cuenta los distintos tipos de gráficas para los distintos tipos de variables:
* Los histogramas sirven para mostrar la distribución de variables numéricas.
* Los gráficos de barras muestran la proporción en los valores de una variable categórica.

In [ ]:
utils.plot_histogram(wisconsin_data_train)

Las variables predictoras de tipo mean y worst que presentan una distribución normal:

* radius_mean aunque es un poco asimétrica. No tiene ruido aunque tiene valores anómalos en 27 y 28.
* texture_mean con un valor anómalo en 39.
* perimeter_mean parece que sigue una distribución normal aunque se encuentra algo asimétrica (aparentemente no tiene ningún valor anómalo).
* smoothness_mean presenta un valor anómalo en 0.16.
* symmetry_mean aparentemente no presenta ningún valor anómalo ni ruidoso.
* fractal_dimension_mean presenta valores anómalos a partir de 0.09. No tiene valores ruidosos.
* radius_worst con un valor anómalo en 36
* texture_worst sin ningún valor anómalo ni ruidoso aparentemente.
* perimeter_worst igual que perimeter_mean presenta una distribución normal aunque se encuentra algo asimétrica. Tiene un valor anómalo en 250.
* smoothness_worst tiene dos valores anómalos en 0.215 y 0.22.
* symmetry_worst con bastantes valores anómalos a partir de 0.5.
* fractal_dimension_worst con valores anómalos a partir de 0.14.

A continuación hablaremos sobre las variables que no representan distribuciones normales:

* concave point_worst y concave point_mean: parecen una mixtura de distribuciones normales. Aparentemente no tienen valores anómalos ni ruidosos.
* area_mean: es parecida a una distribución normal pero no llega a serlo porque sus datos se encuentran balanceados hacia la izquierda. Tiene valores anómalos a partir de 2000.
* area_worst: tiene una distribución parecida a area_mean. Tiene un valor anómalo en 4200.
* compactness_mean y compactness_worst: también tiene una distribución balanceada hacia la izquierda. compactness_mean tiene valores anómalos a partir de 0.27 y compactness_worst a partir de 0.85.
* concavity_mean y concavity_worst: tienen una distribución como las anterior. concavity_mean tiene valores anómalos a partir de 0.4 y concavity_worst a partir de 1.

En las gráficas anteriores no se han encontrado valores ruidosos (no había valores que parecieran erróneos según el significado de la variable) por lo tanto, vamos a analizar las variables de tipo "se" (error estandar) para así poder encontrar los valores ruidosos de las variables:

* radius_se: hay nueve valores que se alejan de la distribución de errores (a partir de 1.2). Estos valores los consideraremos como ruidosos.
* texture_se: hay 14 valores ruidosos a partir de 2.5.
* perimeter_se: tiene dos valores ruidosos a partir de 18.5.
* area_se: tiene cuatro valores ruidosos a partir de 229.99.
* smoothness_se: hay nueve valores ruidosos a partir de 0.015.
* compactness_se: tiene 131 varios valores ruidosos a partir de 0.03
* concavity_se : tiene 216 valores ruidosos a partir de 0.025.
* concave points_se : tiene 105 valores ruidosos a partir de 0.015.
* simmetry_se : tiene 19 valores ruidosos a partir de 0.036.
* fractal_dimension_se : tiene 18 valores ruidosos a partir de 0.0085.

Para analizar la cantidad de valores ruidosos y el umbral por el que consideramos que un valor es ruidoso hemos comparado gráficas de "mean" y "se" para ver los valores promedios. Por ejemplo como concavity tiene valores muy pequeños, el mínimo error estandar ya ocasiona que sea un valor ruidoso

In [ ]:
utils.plot_histogram(pima_data_train)

Los histogramas de algunas variables predictoras representan una distribución normal (BMI, BloodPressure, Glucose, Skin Thickness), aunque presentan algún dato ruidoso ya que son valores erróneos, por ejemplo:

*     BMI, en 0.
*     BloodPressure, en 0.
*     Glucose, en 0.
*     Skin Thickness, en 0, el valor en 95 lo tomaremos como anomalo.

Los datos suelen estar entre los siguientes rangos:

*     BMI se encuentra en el rango [17,69]
*     BloodPressure se encuentra en el rango [20,124]
*     Glucose se encuentra en el rango [55,200]
*     Skin Thickness se encuentra en el rango [4,64]

Por otro lado, las distribuciones de las variables Pregnancies, DiabetesPedigreeFunction son normales pero que tienden a la derecha.

*     La distribución de la variable Pregancies se encuentra en el rango [0,17].
*     La distribución de la variable DiabatesPedrigreeFunction se encuentra en el rango [0,2.5].
*     La distribución de la variable Age se encuentra en el rango [20,71], y encontramos un dato anómalo en el valor 80 porque se aleja de la distribución de la variable.

Respecto a la variable Insuline, podemos encontrar una distribucion normal que tiende a la derecha en el rango [20 - 700], podemos ver unos valores anomalos hasta el 800, los cuales los consideramos anomalos ya que estan muy alejados de la distribucion normal de la variable. Tambien podemos ver una gran cantidad de 0s los cuales los consideramos un valor ruidoso.

Continuamos visualizando las variables categóricas del problema:

In [ ]:
utils.plot_barplot(wisconsin_data_train)

In [ ]:
utils.plot_barplot(pima_data_train)

Lo que podemos apreciar en ambos casos es que el problema está desbalanceado, ya que no tenemos el mismo número de casos para ambos valores. 

A continuación, utilizando un pair_plot, podemos realizar una gráfica por parejas en la que comparamos todas las variables por parejas:

In [ ]:
utils.plot_pairplot(wisconsin_data_train, target="diagnosis")

En el caso de wisconsin, el número de variables es tan elevado que resulta imposible discernir ningún tipo de información útil de esta gráfica, ya que todo resulta confuso.

In [ ]:
utils.plot_pairplot(pima_data_train, target="Outcome")

En el caso de Pima resulta mucho más visible que en wisconsin, pero aún así no somos capaces de tomar ninguna decisión sobre la base de datos partiendo de estas gráficas. 
Por tanto, hemos decidido utilizar otras funciones que describan mejor las variables para, después de estudiarlas más profundamente, realizar una matriz de correlación que nos muestre de forma sencilla cual es la relación entre todas las variables.

In [ ]:
corrM_wisconsin = wisconsin_data_train.corr()

In [ ]:
corrM_wisconsin

In [ ]:
f,ax = mpl.subplots(figsize=(31,31))
sns.heatmap(corrM_wisconsin, annot=True, linewidths= .5, fmt=".1f", ax = ax)

Gracias a este mapa de calor, podemos ver de forma mucho más intuitiva cual es la correlación entre las distintas variables. Nos interesa eliminar las que muestren una correlación muy alta, ya que no aportan demasiada información y complican el proceso.
Por ello, basándonos en este mapa, podemos ver que hay algunas variables que tienen mucha correlación y, por tanto, convendría eliminar.

In [ ]:
corrM_pima = pima_data_train.corr()

In [ ]:
corrM_pima

En el caso de Pima vamos a ver el mapa de correlacion representado con un mapa de calor, aunque al tener menos variables que Wisconsin este mapa de correlacion se puede llegar a entender, pero visualmente es más rapido de entender el mapa de calor.

In [ ]:
f,ax = mpl.subplots(figsize=(31,31))
sns.heatmap(corrM_pima, annot=True, linewidths= .5, fmt=".1f", ax = ax)

Igual que hemos dicho antess gracias a este mapa de colores podemos ver mejor la correlacion que hay entre las variables de nuestra base de datos.

Tras ver las variables más correlacionadas, vamos a mostrar ahora una gráfica para cada conjunto de datos en las que se muestra si alguna de ellas tiene algun valor nulo.

In [ ]:
utils.missingdata(wisconsin_data_train)

En el caso de wisconsin podemos ver que todas las variables carecen de valores nulos salvo por la variable unnamed 32, que solo tiene valores nulos.

In [ ]:
utils.missingdata(pima_data_train)

En el caso de pima, no encontramos ninguna variable con valores nulos.

Ahora vamos a ver la cantidad de ceros que tienen las variables de pima, y que no tendrian sentido estos resultados.

Glucose, BloodPressure, SkinThickness, Insulin y BMI es imposible que tengan un valor 0 biologicamente y podria ser que los valores nulos han sido codificados como ceros en estos casos

In [ ]:
print("Pregnances:",(pima_data_train.Pregnancies == 0).sum())
print("DiabetesPedigreeFunction:",(pima_data_train.DiabetesPedigreeFunction==0).sum())
print("Age:",(pima_data_train.Age==0).sum())

print("Glucose:",(pima_data_train.Glucose==0).sum())
print("BloodPressure:",(pima_data_train.BloodPressure==0).sum())
print("SkinThickness:",(pima_data_train.SkinThickness==0).sum())
print("Insulin:",(pima_data_train.Insulin==0).sum())
print("BMI:",(pima_data_train.BMI==0).sum())



Ahora vamos convertir estos 0s en 'Nan' en las variables donde no tienen sentido estos 0s es decir en Glucose, BloodPressure, SkinThickness, Insulin y BMI.

In [ ]:
pima_data_train = pima_data_train.replace({'Glucose': 0,'BloodPressure': 0,'SkinThickness': 0,'Insulin': 0,'BMI': 0},np.nan)



Ahora vamos a volver a mostrar el conjunto de datos de Diabetes para ver los valores a Nan

In [ ]:
utils.missingdata(pima_data_train)

Podemos observar que Insulin tiene un 48.6% de valores Nulos (anteriormente 0s), este es un porcentaje muy alto por lo que deberiamos eliminar esta variable. SkinThickness igualmente tiene un 29.2% de valores nulos, no es tan alto como insulin pero tambien es un porcentaje considerable por lo que tambien desecharemos esta variable. Respeccto a BloodPressure, BMI y Glucose vemos que hay algunos casos pero que no llegan ni al 5% asi que no eliminaremos estas variables y estos casos seran tratados.

A continuación, utilizaremos un diagrama de caja para ver los intervalos estándar de las variables y definir los límites por los que se define un valor anómalo.

In [ ]:
utils.plot_box_diagram(wisconsin_data_train)

Gracias a este diagrama obtenemos datos más precisos de nuestras variables y además podemos observar los valores anomalos que toman nuestras variables.

###### Datos Mean:

    Radius:[6.981 - 22.27] -> valores anomalos entre el 23.21 y el 28.11
    Texture:[10.38 - 29.97] -> valores anomalos entre el 30.72 y el 39.28
    Perimeter:[43.79 - 152.1] -> valores anomalos entre el 152.8 y el 188.5
    Area:[143.5 - 1364] -> valores anomalos entre el 1384 y el 2501
    Smoothness:[0.06251 - 0.1326] -> valores anomalos entre el 0.1371 y el 0.1634
    Compactness:[0.01938 - 0.2293] -> valores anomalos entre el 0.2363 y el 0.3454
    Concavity:[0 - 0.2871] -> valores anomalos entre el 0.3001 y el 0.4268
    Concave_Points:[0 - 0.152] -> valores anomalos entre el 0.1595 y el 0.2012
    Symmetry:[0.1203 - 0.2459] -> valores anomalos entre el 0.2495 y el 0.304, y el 0.106
    Fractal_Dimension:[0.04996 - 0.0795] -> valores anomalos entre el 0.08046 y el 0.09744
    
###### Datos SE:

    Radius:[0.1115 - 0.9317] -> valores anomalos entre el 0.9553 y el 2.873
    Texture:[0.3981 - 2.342] -> valores anomalos entre el 2.426 y el 4.885
    Perimeter:[0.7714 - 6.372] -> valores anomalos entre el 6.462 y el 21.98
    Area:[7.228 - 90.47] -> valores anomalos entre el 93.91 y el 542.2
    Smoothness:[0.001713 - 0.01215] -> valores anomalos entre el 0.01236 y el 0.03113
    Compactness:[0.00252 - 0.06158] -> valores anomalos entre el 0.06213 y el 0.1354
    Concavity:[0 - 0.08232] -> valores anomalos entre el 0.09263 y el 0.1535
    Concave_Points:[0 - 0.02593] -> valores anomalos entre el 0.02598 y el 0.05279
    Symmetry:[0.01013 - 0.03546] -> valores anomalos entre el 0.03756 y el 0.07895
    Fractal_Dimension:[968.3µ - 0.007877] -> valores anomalos entre el 0.008015 y el 0.02984
    
###### Datos Worst:

    Radius:[7.93 - 28.4] -> valores anomalos entre el 29.17 y el 36.04
    Texture:[12.49 - 41.85] -> valores anomalos entre el 44.87 y el 47.16
    Perimeter:[50.41 - 188.5] -> valores anomalos entre el 195 y el 251.2
    Area:[185.2 - 2022] -> valores anomalos entre el 2053 y el 4254
    Smoothness:[0.08125 - 0.1862] -> valores anomalos entre el 0.1873 y el 0.2226 y el 0.07117
    Compactness:[0.02729 - 0.6247] -> valores anomalos entre el 0.659 y el 1.058
    Concavity:[0 - 0.8216] -> valores anomalos entre el 0.8402 y el 1.252
    Concave_Points:[0 - 0.291] -> sin valores anomalos
    Symmetry:[0.1566 - 0.4228] -> valores anomalos entre el 0.4264 y el 0.6638
    Fractal_Dimension:[0.05521 - 0.1224] -> valores anomalos entre el 0.1233 y el 0.2075
    
    
    

Repetimos el proceso para el conjunto Pima.

In [ ]:
utils.plot_box_diagram(pima_data_train)

    Pregnances:[0 - 13] -> valores anomalos 14 y 17
    Glucose:[56 - 199] -> valor anomalo 0
    BloodPressure:[38 - 102] -> valores anomalos 0, y entre el 104 y el 122
    SkinThickness:[1 - 63] -> valores anomalos 99 y el 0 o muy cercanos ya que es imposible que el grosor de la piel sea 0
    Insulin:[14 - 342] -> valores anomalos entre el 318 , el 510 y el 0, los mayores podrían ser considerados ruido (543, 579, 600, 680, 846)
    BMI:[18.2 - 50] -> valores anomalos 0, 52.3, 52.9, 53.2, 57.3, 59.4
    DiabetesPedigreeFunction:[0.078 - 1.182] -> valores anomalos entre el 1.213 y el 2.42
    Age:[21 - 64] -> valores anomalos entre el 65 y el 81

# 4. Preprocesamiento de datos

Este paso del procedimiento es fundamental, pues es el que más va a influenciar nuestra eficacia en el proceso. Aquí, modificaremos nuestro conjunto de datos para que se adapte a nuestras necesidades y ofrezca mejores resultados en el entrenamiento y el test.
Vamos a seguir los siguientes pasos:

* Limpieza de datos: donde trataremos los valores perdidos, ruidosos...
* Discretización: a través de selección de variables, de instancias...

### Limpieza de datos

Una vez tenemos los intervalos de valores para las variables, creamos transformadores para suavizar los valores anomalos:

Con toda la información que hemos obtenido ( correlacion entre variables, variables ruidosas, valores anomalos) vamos a descartar las variables 'concavity', 'compactness' y 'concavity points', ya que son variables que tienen mucha correlación con otras variables y además tienen mucho ruido. Tambien vamos a eliminar perimeter y area por su grán parecido a radius (mucha correlacion). Además, quitaremos las variables de tipo 'SE' y las de tipo 'WORST' ya que no se deben de usar para predecir.

In [ ]:
del_columns_wisconsin = utils.QuitarColumnasTransformer(['perimeter_mean', 'area_mean', 'compactness_mean', 'concave points_mean','radius_se','texture_se',
                                    'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
                                    'fractal_dimension_se', 'radius_worst', 'smoothness_worst', 'symmetry_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 
                                    'compactness_worst', 'concavity_worst', 'concave points_worst', 'fractal_dimension_worst', 'Unnamed: 32'])

nan_anomalos_wisconsin=utils.AnomalosANanTransformer({'radius_mean':[6.981,22.27], 'texture_mean':[10.38,29.97],'smoothness_mean':[0.06251, 0.1326],
                                     'symmetry_mean':[0.1203, 0.2459], 'fractal_dimension_mean':[0.04996, 0.0795], 'concavity_mean':[0, 0.2871]})

Con Pima quitaremos 'SkinThickness' e 'Insulin' debido a la gran cantidad de valores 'Nan' o anteriormente 0s, tambien pudimos observar gracias al mapa de correlacion que Age y Pregnances tienen bastante correlacion entre ellas (0.6) pero no vemos necesario eliminar alguna de ellas.

In [ ]:
del_columns_pima = utils.QuitarColumnasTransformer(['SkinThickness', 'Insulin'])

nan_anomalos_pima=utils.AnomalosANanTransformer({'Glucose':[56,199], 'BloodPressure':[38,102], 'Pregnancies':[0,13],
                                      'BMI':[18.2,50], 'DiabetesPedigreeFunction':[0.078,1.182], 'Age':[21,64], })

Después de crear los transformadores referentes a los datos anomalos y variables que no vamos a utilizar para nuestro pipeline, ahora creamos un imputador donde todos los valores nulos seran sustituidos por la media.

In [ ]:
imp = SimpleImputer(missing_values=float('nan'), strategy='mean')

### Discretización

El proceso de discretización permite convertir variables numéricas en categóricas. Esto facilita mucho el trabajo para los algoritmos de aprendizaje.
Nosotros crearemos un discretizador para cada conjunto de datos.

Antes de nada vamos a volver a generar unos diagramas de nube de puntos (pairplots) que nos ayudaran a generar nuestro discretizador facilitandonos la decision sobre la cantidad de conjuntos en los que dividir el rango de nuestras variables.

In [ ]:
wisconsin_data_train_aux=wisconsin_data_train[['radius_mean','texture_mean','smoothness_mean','symmetry_mean','fractal_dimension_mean','concavity_mean','diagnosis']]

In [ ]:
utils.plot_pairplot(wisconsin_data_train_aux, target="diagnosis")

En el pairplot podemos ver que si dividimos los rangos en 5 hay partes donde encontraremos solo puntos azules en la izquierda y puntos rojos en la derecha de algunos diagramas.
Por lo que decidimos que nuestro discretizador tendra 5 conjuntos.

In [ ]:
discretizer_wisconsin = KBinsDiscretizer(n_bins=5, strategy="uniform")

In [ ]:
pima_data_train_aux=pima_data_train[['Pregnancies','Glucose','BloodPressure','BMI','DiabetesPedigreeFunction','Age','Outcome']]

In [ ]:
utils.plot_pairplot(pima_data_train_aux, target="Outcome")

In [ ]:
discretizer_pima = KBinsDiscretizer(n_bins=5, strategy="uniform")

El discretizador de pima utiliza también la estrategia uniforme y define 5 conjuntos de la misma forma que anteriormente.

# 5. Algoritmos de clasificación

El primer algoritmo de clasificación que vamos a aprender es el más sencillo de todos, el Zero-R.

### Algoritmo *Zero-R*

El clasificador ZeroR predice simplemente la categoría mayoritaria. Aunque no hay poder de predictibilidad en ZeroR, es útil para determinar un desempeño como un punto de referencia para otros métodos de clasificación.

In [ ]:
zero_r_model = DummyClassifier(strategy="most_frequent")

### Algoritmo *CART* (*Classification and Regression Trees*): Inducción de árboles de decisión

El algoritmo CART es el acrónimo de Classification And Regression Trees. Con este algoritmo, se generan árboles de decisión binarios, lo que quiere decir que cada nodo se divide en exactamente dos ramas.

Este modelo admite variables de entrada y de salida nominales, ordinales y continuas, por lo que se pueden resolver tanto problemas de clasificación como de regresión.

In [ ]:
tree_model_classifier = DecisionTreeClassifier(random_state=seed)

### *Pipeline*

Antes de evaluar los modelos, vamos a crear los pipeline, que utilizaremos para aplicar los transformadores que hemos creado anteriormente al conjunto de datos.
Crearemos los pipeline con la función make_pipeline y le pasaremos como parámetros los transformadores y el estimador.

## Zero-R Model Pipelines

In [ ]:
wisconsin_zero_r_model = make_pipeline(del_columns_wisconsin, nan_anomalos_wisconsin, imp, zero_r_model)

In [ ]:
pima_zero_r_model = make_pipeline(del_columns_pima, nan_anomalos_pima, imp, zero_r_model)

## Tree Model Pipelines

Sin discretizador

In [ ]:
wisconsin_tree_model = make_pipeline(del_columns_wisconsin, nan_anomalos_wisconsin, imp, tree_model_classifier)

In [ ]:
pima_tree_model = make_pipeline(del_columns_pima, nan_anomalos_pima, imp, tree_model_classifier)

Con discretizador

In [ ]:
wisconsin_discretize_tree_model = make_pipeline(del_columns_wisconsin, nan_anomalos_wisconsin, imp, discretizer_wisconsin, tree_model_classifier)

In [ ]:
pima_discretize_tree_model = make_pipeline(del_columns_pima, nan_anomalos_pima, imp, discretizer_pima, tree_model_classifier)

# 6. Evaluación de modelos

Por último, vamos a evaluar el rendimiento de los modelos creados usando una matriz de confusión y la tasa de acierto asociada.

In [ ]:
utils.evaluate(wisconsin_zero_r_model,
               X_wisconsin_train, X_wisconsin_test,
               y_wisconsin_train, y_wisconsin_test,'M')

In [ ]:
utils.evaluate2(pima_zero_r_model,
               X_pima_train, X_pima_test,
               y_pima_train, y_pima_test)

Evidentemente, los resultados del Zero-R son nulos porque se trata de un algoritmo muy poco sofisticado que se limita a predecir siempre el valor más común de la variable objetivo. Podemos observar que en el caso de Cancer con este algoritmo siempre predice como Benigno y esto no se puede tener en cuenta, de igual forma para diabetes ya que siempre predice que no hay diabetes (0).


In [ ]:
utils.evaluate(wisconsin_tree_model,
               X_wisconsin_train, X_wisconsin_test,
               y_wisconsin_train, y_wisconsin_test,'M')

In [ ]:
utils.evaluate2(pima_tree_model,
               X_pima_train, X_pima_test,
               y_pima_train, y_pima_test)

Y con el conjunto de datos discretizado:

In [ ]:
utils.evaluate(wisconsin_discretize_tree_model,
               X_wisconsin_train, X_wisconsin_test,
               y_wisconsin_train, y_wisconsin_test,'M')

In [ ]:
utils.evaluate2(pima_discretize_tree_model,
               X_pima_train, X_pima_test,
               y_pima_train, y_pima_test)

El modelo de árbol de decisión ofrece un rendimiento muy superior al Zero-R, ya que se trata de un algoritmo mucho más elaborado y utiliza de forma más eficiente los datos (con un 0,84 para wisconsin y un 0,59 para pima sin discretizadoy 0.75 en wisconsin y 0.56 en pima con el discretizado).Estos resultados no son los peores pero ni mucho menos los mejores dado que en ningun caso hemos llegado ni si quiera a una tasa de acierto del 90%, nosotros nos esperabamos que al utilizar el discretizado la tasa de acierto iba a mejorarpero al contrario esta empeoro en casi un 10% para cancer y en casi un 5% para diabetes.

# 7.Titanic

Por último, como parte final de la entrega Extraordinaria, vamos a añadir la base de datos Titanic y vamos a proceder a explorarla como las dos bases de datos anteriores. Este conjunto de datos tiene como idea ofrecer algunas características de los pasajeros abordo del famoso transatláctico y, a partir de las mismas, debemos predecir si ese pasajero sobrevive o no a su hundimiento.

Las variables que encontramos en este conjunto de datos son:

* Survival: variable clase con valor 1 si sobrevive o 0 si no lo hace.
* Pclass: clase del ticket. Puede ser 1, 2 o 3.
* Sex: sexo del pasajero. "Male" o "Female".
* Age: edad del pasajero en años.
* Sibsip: número de parejas y hermanos a bordo.
* Parch: número de padres e hijos a bordo.
* Ticket: número del ticket.
* Fare: tarifa del pasajero.
* Cabin: número de cabina del pasajero.
* Embarked: puerto de embarque. (C para Cherbourg, Q para Queenstown y S para Southampton).

Una vez conocemos la base de datos, vamos a proceder a incluirla en la libreta utilizando su filepath. Indicaremos el índice del conjunto utilizando la variable PassengerId y la variable objetivo con Survived.

In [ ]:
filepath = "../input/titanic/train.csv"

index = "PassengerId"
target = "Survived"

titanic_data = utils.load_data(filepath, index, target)

Usamos el comando sample para comprobar que se ha cargado adecuadamente.

In [ ]:
titanic_data.sample(5)

Podemos ver ahora el número de variables que hay y su tipo.

In [ ]:
titanic_data["Survived"]=titanic_data["Survived"].astype("category")
titanic_data.info(memory_usage=False)

Tenemos 10 variables. Cinco de ellas son numéricas, tanto enteros como float (Pclass, Age, Sibsip, Parch y Fare). Sin embargo, hay otras de tipo object que debemos modificar para poder utilizarlas correctamente (Name, Sex, Ticket, Cabin y Embarked).

Después de haber estudiado las variables, vamos a dividir el conjunto de datos en variables predictoras (X) y variable objetivo (y) para hacer más cómodas algunas operaciones.

In [ ]:
(X_titanic_data,y_titanic_data)=utils.divide_dataset(titanic_data,target="Survived")

In [ ]:
X_titanic_data.sample(5,random_state=seed)

In [ ]:
y_titanic_data.sample(5,random_state=seed)

Además, vamos a dividir el conjunto X e Y a su vez en conjuntos de entrenamiento y test, utilizando la proporción de antes y estratificación.

In [ ]:
train_size = 0.7

(X_titanic_data_train, X_titanic_data_test, y_titanic_data_train, y_titanic_data_test) = train_test_split(X_titanic_data, y_titanic_data,
                                                      stratify=y_titanic_data,
                                                      random_state=seed,
                                                      train_size=train_size)

In [ ]:
X_titanic_data_train.sample(5,random_state=seed)

In [ ]:
X_titanic_data_test.sample(5,random_state=seed)

In [ ]:
y_titanic_data_train.sample(5,random_state=seed)

In [ ]:
y_titanic_data_test.sample(5,random_state=seed)

Ahora que tenemos el conjunto entrenamiento de X e Y en una variable, vamos a volver a unirlos en otra variable para agilizar las operaciones y tener el conjunto completo de entrenamiento.

In [ ]:
titanic_data_train=utils.join_dataset(X_titanic_data_train,y_titanic_data_train)

In [ ]:
titanic_data_train.sample(5,random_state=seed)

Volvemos a unir también los dos subconjuntos de test que habíamos separado para poder volver a trabajar con el conjunto completo.

In [ ]:
titanic_data_test=utils.join_dataset(X_titanic_data_test,y_titanic_data_test)

In [ ]:
titanic_data_test.sample(5,random_state=seed)

Ahora, con el conjunto de entrenamiento completo, podemos estudiarlo a fondo antes de comenzar el preprocesamiento de datos.

In [ ]:
titanic_data_train.shape

Como nos indica la función shape, tenemos un total de 623 pasajeros y 11 variables en el conjunto de entrenamiento.

In [ ]:
y_titanic_data_train.cat.categories

La variable categorica podemos ver que toma valores enteros 1 y 0.

Vamos a estudiar las variables utilizando una gráfica de barras que indique la distribución de los valores de cada variable.

In [ ]:
utils.plot_histogram(titanic_data_train)

Si nos fijamos en la gráfica, podemos llegar a varias conclusiones:
* En la gráfica no aparecen todas las variables, solo las que tienen valores numéricos.
* La gran mayoría de las variables tienden a los extremos y ofrecen una distribución desbalanceada a la izquierda.

Por ello, resulta evidente que debemos procesar los datos para poder utilizarlos adecuadamente.

Respecto a los datos mostrados en la grafica podemos observar:

* Pclass la mayoria de gente son de tercera clase, de primera y de segunda hay mas o menos la misma cantidad de gente.
* Age podemos observar que la mayoria de personas rondan entre los 20 y 30 años, y tiene una distribucion normal.
* SibSp respecto a familiares vemos que la mayoria de personas han ido sin familiares del tipo hermanos, esposa o esposo, y tiene una distribucion normal que tiende a la derecha.
* Parch en cuanto a padres o hijos tambien vemos que la mayoria de gentes ha ido sin padres o hijos, igual a SibSp tambien tiene una distribucion que tiende a la derecha.
* Fare podemos  ver que la tarifa mayormente utilizada por la gente esta entre 5 y 14.9, podemos ver valores anomalos como el valor cercano a 500, aunque podria llegar a ser verdad, tambien tiene una distribucion normal que tiende a la derecha pero en este caso podemos encontrar algunos valores anomalos por la distancia a la distribucion normal, como los que van desde el 205 al 514.9.

Vamos a comprobar también la proporción de valores de la variable objetivo.

In [ ]:
utils.plot_barplot(titanic_data_train)

Como podemos ver, el problema está desbalanceado porque el número de los que sobreviven y los que no no es similar.

Mostramos una gráfica que nos indique el número de valores nulos que tiene cada variable. Vemos claramente que cabin tiene una proporción enorme y age tiene una cantidad que no llega al 20%. Por tanto, deberíamos eliminar cabin unicamente por la cantidad de valores nulos que tiene.

In [ ]:
utils.missingdata(titanic_data_train)

In [ ]:
copiaT=titanic_data_train.copy()
copiaT = copiaT.drop(['Name', 'Ticket', 'Cabin', 'Fare'], axis=1)
copiaT['Sex']=copiaT['Sex'].map({'male':1,'female':0})
copiaT['Embarked']=copiaT['Embarked'].map({'Q':1,'C':2,'S':3})

Eliminamos Nombre, Ticket, Cabin y Fare porque son datos innecesarios para saber si sobrevivieron los pasajeros. y Por el numero de valores nulos que tiene Cabin.

In [ ]:
utils.plot_pairplot(copiaT, target="Survived")

Realizamos también una gráfica por parejas en la que se nos muestra la relación entre cada pareja de variables. Sin emabargo, resulta tan caótico al final, que es muy difícil extraer ningún tipo de conclusión. Por ello, como en los conjuntos anteriores, vamos a utilizar una matriz de correlación que nos muestre más claramente la relación entre las variables.

In [ ]:
corrMatrix = copiaT.corr()

In [ ]:
corrMatrix

In [ ]:
f,ax=mpl.subplots(figsize=(7,7))
sns.heatmap(corrMatrix,annot=True,linewidths=.5,fmt=".1f",ax=ax)

Utilizando un mapa de calor para ilustrar la matriz de correlación, vemos que no hay ningún par de variables que tenga una correlación alta o, ni siquiera, reseñable. De hecho, en algunos casos vemos una correlación negativa, aunque nunca extrema.

In [ ]:
utils.plot_box_diagram(copiaT)

Por último, antes de crear el pipeline, hemos utilizado un diagrama de cajas que nos muestra el rango de valores que suelen tener las variables numéricas. Con esta información, podemos delimitar rangos de valores para las variables numéricas, tanto las que son numéricas inicialmente como las que eran categoricas las cuales hemos hecho numericas tambien para una mas facil representacion y tratamiento.

    Pclass: [1 - 3] -> como vimos anteriormente solo tenemos estas tres clases y no se ven valores anomalos.
    Sex: [0 - 1] -> es una variable binaria donde solo se puede ser hombre (1) o mujer (0), no se encuentran valores anomalos.
    Age: [20 - 39] -> vemos que hay algunos valores fuera de lo comun como recien nacidos con 0 años y personas ancianas con hasta 80 años, esto es menos comun pero no lo trataremos como valores anomalos.
    SibSp: [0 -1] -> la mayoria de gente ha venido sin o con un nunico acompañante cercano (marido, mujer, hermano) aunque hay casos extraños como el 3, 4, 5 y el 8.
    Parch: [0-2] -> la mayoria de gene ha venido sola o con un hijo/padre, hay pocos que han venido con dos, y podemos ver casos pocos comunes que han venido 3, 4, 5, o hasta 6 acompañantes entre hijos y padres.
    Fare: [7.92 - 30.6] -> respecto a las tarifas la mayoria tienen una qure ronda el precio de entre los 8 a los 3, aunque podemos encontrar muchos valores anomalos (puede ser resultado de que fueran los ultimos tickets o una posible reventa) que llegan hasta los 500.
    Embarked: [1 - 3] -> unicamente hay tres posibilidades y no encontramos valores anomalos.

Utilizaremos los siguientes transformadores para generar nuestro Pipeline:

* del_columns: la utilizamos para borrar las variables que no aportan información o tienen muchos valores anomalos (cabin, name, ticket y fare).
* Usamos cambiarStringAInt para convertir Sex y Embarked a un rango de números enteros que se corresponden con sus clases.
* nan_anomalos_titanic sirve para definir un rango de valores para cada variable, evitando así lo anómalos.
* Imputador simple, que intercambia los valores nulos por los valores más utilizados ya que en algunos casos las  variables son categoricas y no se puede utilizar la media.
* Discretizador de dos bins y estrategia uniforme.

In [ ]:
del_columns = utils.QuitarColumnasTransformer(['Cabin', 'Name', 'Ticket', 'Fare'])

string_int1 = utils.StringAIntTransformer('Sex',['female','male'],[0,1])
string_int2 = utils.StringAIntTransformer('Embarked',['Q','C','S'],[1,2,3])

nan_anomalos_titanic = utils.AnomalosANanTransformer({'Pclass':[1,3], 'Sex':[0,1], 'SibSp':[0, 8],
                                         'Parch':[0, 6], 'Embarked':[1, 3] })

imp = SimpleImputer(missing_values=float('nan'), strategy='most_frequent')

discretizer_titanic = KBinsDiscretizer(n_bins=2, strategy="uniform")

Ahora vamos a crear los pipelines con los transformadores creados anteriormente.

In [ ]:
titanic_zero_r_model = make_pipeline(del_columns,string_int1,string_int2,nan_anomalos_titanic, imp, zero_r_model)

In [ ]:
titanic_tree_model = make_pipeline(del_columns,string_int1,string_int2,nan_anomalos_titanic, imp, tree_model_classifier)

In [ ]:
titanic_tree_model_disc = make_pipeline(del_columns,string_int1,string_int2,nan_anomalos_titanic, imp, discretizer_titanic, tree_model_classifier)

Una vez hemos creado el modelo pasando el pipeline al conjunto de datos, vamos a estudiar los resultados de cada uno:

In [ ]:
utils.evaluate(titanic_zero_r_model,
               X_titanic_data_train, X_titanic_data_test,
               y_titanic_data_train, y_titanic_data_test)

El modelo Zero-R obtiene resultados nulos, al igual que los dos realizados anteriormente con las otras bases de datos. Al igual que anteriormente explicado esto se bebe a que siempre elige la opcion mayoritaria aunque no sea la correcta.

In [ ]:
utils.evaluate(titanic_tree_model,
               X_titanic_data_train, X_titanic_data_test,
               y_titanic_data_train, y_titanic_data_test)

El modelo con árbol de decisión mejora notablemente los resultados ya que, utiliza un modelo más elaborado. Por ello, obtenemos hasta un 0.660 de recall en este caso, que aunque nu es un muy buen porcentaje tampoco es el peor.

In [ ]:
utils.evaluate(titanic_tree_model_disc,
               X_titanic_data_train, X_titanic_data_test,
               y_titanic_data_train, y_titanic_data_test)

Sin embargo, en el modelo de árbol de decisión discretizado no encontramos ninguna mejora, más que mejorar nos empeora los resultados, ya que obtenemos un 0.57. Seguramente esta influyendo el hecho de que los datos recibidos son bastante caóticos y no ofrecen muchas posibilidades a la hora de realizar una predicción.